# Import Libraries

In [120]:
import pandas as pd
import numpy as np
import h5py
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
import string as st
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import torch
import fasttext
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

# Linguistic Modules

In [121]:
def preprocess_text(text):
    doc = nlp(text.lower().strip())  # Lowercase and remove whitespace
    
    # Remove stopwords, punctuation, and lemmatize
    tokens = [token.lemma_ for token in doc 
             if not token.is_stop and not token.is_punct and token.is_alpha]
    
    return ' '.join(tokens)

In [122]:
def extract_linguistic_features(text):
    doc = nlp(text)
    features = {
        'num_verbs': len([token for token in doc if token.pos_ == 'VERB']),
        'num_nouns': len([token for token in doc if token.pos_ == 'NOUN']),
        'sentence_length': len(doc),
        'blooms_verb_present': any(token.text in {'analyze', 'evaluate', 'create'} for token in doc)
    }
    return features

In [123]:
def get_sentence_vector(text):
    words = text.strip().split()
    vectors = [model.get_word_vector(w) for w in words]
    if not vectors:
        return np.zeros(300)
    return np.mean(vectors, axis=0)

# Pre-Processing

### Import Data

In [131]:
dataset_name = 'balanced_dataset'

# Load dataset
df = pd.read_csv(os.getcwd().replace('notebook' , 'dataset') + '/' + dataset_name + '.csv')

# Apply preprocessing
df['processed_question'] = df['QUESTION'].apply(preprocess_text)

## Tokenize

BERT model is used to retain context

In [45]:
# Load pre-trained BERT (TensorFlow version)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode
inputs = tokenizer(df['QUESTION'].tolist(), return_tensors='tf', padding=True, truncation=True)
outputs = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]
print("BERT Sentence Embedding Shape:", sentence_embedding.shape)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

BERT Sentence Embedding Shape: (600, 768)


In [ ]:
pca = PCA(n_components=100)
reduced_embeddings = pca.fit_transform(sentence_embedding.numpy())


### TF-IDF 
Execute if either BERT or IF-IDF

In [132]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['processed_question'])

In [133]:
# Convert TF-IDF sparse matrix to DataFrame with appropriate column names
tfidf_df = pd.DataFrame(tfidf.toarray(), columns= vectorizer.get_feature_names_out())

# Adding linguistic features
ling_features = df['processed_question'].apply(extract_linguistic_features).apply(pd.Series)

# Concatenate DataFrames
token_df = pd.concat([ling_features, tfidf_df], axis=1)

# Drop unnecessary columns (if needed)
token_df = token_df.drop(columns=['blooms_verb_present'], axis=1)
token_df.columns = token_df.columns.map(str)

token_df = pca.fit_transform(token_df)

# Modelling

## Basic Machine Leanring

In [134]:
x = tfidf_df
y = df['BT LEVEL']

In [135]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=34 , stratify= y)

### Random Forest

In [142]:
params = {
    'n_estimators' : [2 , 5 , 10 , 20 , 50 , 100],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 5 , 10 , 20 , 50 , 100 , 200 , 350 , 500]
}
param_search = GridSearchCV(estimator= RandomForestClassifier(), param_grid= params, cv=5)

param_search.fit(x , y)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
35,0.730000,"{'criterion': 'gini', 'max_depth': 100, 'n_estimators': 100}"
28,0.730000,"{'criterion': 'gini', 'max_depth': 50, 'n_estimators': 50}"
4,0.725000,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 50}"
23,0.720000,"{'criterion': 'gini', 'max_depth': 20, 'n_estimators': 100}"
5,0.718333,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 100}"
34,0.718333,"{'criterion': 'gini', 'max_depth': 100, 'n_estimators': 50}"
29,0.718333,"{'criterion': 'gini', 'max_depth': 50, 'n_estimators': 100}"
40,0.716667,"{'criterion': 'gini', 'max_depth': 200, 'n_estimators': 50}"
52,0.713333,"{'criterion': 'gini', 'max_depth': 500, 'n_estimators': 50}"
57,0.711667,"{'criterion': 'entropy', 'max_depth': None, 'n_estimators': 20}"


In [143]:
classifier = RandomForestClassifier(n_estimators= 100 , max_depth= 100 , criterion= 'gini')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     Analysis       0.91      1.00      0.95        20
  Application       0.83      0.75      0.79        20
Comprehension       0.94      0.80      0.86        20
   Evaluation       0.88      0.70      0.78        20
    Knowledge       0.58      0.90      0.71        20
    Synthesis       0.88      0.70      0.78        20

     accuracy                           0.81       120
    macro avg       0.84      0.81      0.81       120
 weighted avg       0.84      0.81      0.81       120



In [ ]:
question = 'How many total disk access is needed to search a record using two level indexing?'
processed_q = preprocess_text(question)
tfidf_q = vectorizer.transform([processed_q])
tfidf_q = pd.DataFrame(tfidf_q.toarray(), columns=vectorizer.get_feature_names_out())



# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, tfidf_q], axis=1)
token_q = token_q.drop(columns= ['blooms_verb_present'])

print(classifier.predict(tfidf_q))

['Evaluation']


In [ ]:
question = 'How many total disk access is needed to search a record using two level indexing?'
inputs =  tokenizer([question], return_tensors='tf', padding=True, truncation=True)
outputs = model(**inputs)

# Extract embeddings (CLS token for sentence representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]